### LLMChains & OutputParsers

Instead of just using models you can combine a model and a prompt. This can be done with the LCEL (LangChain Expression Language) approach using the pipe operator.

We will see additional, more complex chains in this notebook

In [15]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [16]:
TEMPLATE = """
Interprete the text and evaluate the text.

sentiment: is the text in a positive, neutral or negative sentiment? Sentiment is required.

subject: What subject is the text about? Use exactly one word. Use 'None' if no subject was provided.

price: How much did the customer pay? Use 'None' if no price was provided.

Format the output as JSON with the following keys:
sentiment
subject
price

text: {input}
"""

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

prompt_template = ChatPromptTemplate.from_template(template=TEMPLATE)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({"input": "I ordered pizza salami from the restaurant Bellavista. It was ok, but the dough could have been a bit more crisp."})

'```json\n{\n  "sentiment": "negative",\n  "subject": "pizza",\n  "price": "None"\n}\n```'

### Response Schemas

There were two issues with the output: The output also contains text and the output is just a string, not a dictionary.

In [19]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

# Define the output schema using Pydantic
class ReviewOutput(BaseModel):
    sentiment: str = Field(description="is the text in a positive, neutral or negative sentiment? Sentiment is required.")
    subject: str = Field(description="What subject is the text about? Use exactly one word. Use None if no subject was provided.")
    price: float = Field(description="How much did the customer pay? Use None if no price was provided.", default=None)

# Create the JSON parser with the schema
parser = JsonOutputParser(pydantic_object=ReviewOutput)

# Get format instructions
format_instructions = parser.get_format_instructions()
print(format_instructions)

STRICT OUTPUT FORMAT:
- Return only the JSON value that conforms to the schema. Do not include any additional text, explanations, headings, or separators.
- Do not wrap the JSON in Markdown or code fences (no ``` or ```json).
- Do not prepend or append any text (e.g., do not write "Here is the JSON:").
- The response must be a single top-level JSON value exactly as required by the schema (object/array/etc.), with no trailing commas or comments.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]} the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema (shown in a code block for readability only — do not include any backticks or Markdown in your output):


In [20]:
# Create prompt template with format instructions using modern tuple format
from langchain_core.prompts import ChatPromptTemplate

# Create a proper chat prompt with system and user messages
prompt = ChatPromptTemplate.from_messages([
    ("system", "Interprete the text and evaluate the text. "
               "sentiment: is the text in a positive, neutral or negative sentiment? "
               "subject: What subject is the text about? Use exactly one word. "
               "Just return the JSON, do not add ANYTHING, NO INTERPRETATION!"),
    ("human", "{format_instructions}\n\ntext: {input}")
]).partial(format_instructions=format_instructions)

In [21]:
# Create the chain with the parser
chain = prompt | llm | parser

# Invoke the chain - now only needs 'input' since format_instructions is already bound
result = chain.invoke({"input": "I ordered pizza salami from the restaurant Bellavista. It was ok, but the dough could have been a bit more crisp."})
print(result)

{'sentiment': 'neutral', 'subject': 'pizza', 'price': None}


In [22]:
# Access individual fields
print("Sentiment:", result.get("sentiment"))
print("Subject:", result.get("subject"))
print("Price:", result.get("price"))

Sentiment: neutral
Subject: pizza
Price: None
